In [1]:
import torch
import torch.nn.functional as F
from torchvision import transforms
import os
import tifffile as tiff
import torchvision.transforms.functional as TF
import numpy as np


def preprocess_images_from_directory(image_dir):
    """
    Load and preprocess all 3-layer TIFF images in a directory with deterministic augmentations.

    Args:
        image_dir (str): Path to the directory containing TIFF images.

    Returns:
        dict: Dictionary containing original, horizontally flipped, vertically flipped,
              sharpness-adjusted, blurred, and brightness-adjusted images.
    """
    sharpness_factor = 2.0
    gaussian_blur = transforms.GaussianBlur(kernel_size=5, sigma=1)

    augmentations = {
        "original": [],
        "hori": [],
        "veri": [],
        "sharpness": [],
        "blur": [],
        "brightness_reduced": [],
        "brightness_increased": []
    }

    image_paths = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith(('.tiff', '.tif'))]

    for image_path in image_paths:
        try:
            image = tiff.imread(image_path)

            if image.shape[0] != 3:
                raise ValueError(f"Image at {image_path} does not have exactly 3 layers.")

            image = image.astype(np.float32) / 65535.0
            image = torch.tensor(image, dtype=torch.float32)
            image = TF.resize(image, (96, 96))

            augmentations["original"].append(image.unsqueeze(0))
            augmentations["hori"].append(torch.flip(image, dims=[2]).unsqueeze(0))
            augmentations["veri"].append(torch.flip(image, dims=[1]).unsqueeze(0))
            sharpened_image = TF.adjust_sharpness(image, sharpness_factor=sharpness_factor)
            augmentations["sharpness"].append(sharpened_image.unsqueeze(0))
            blurred_image = gaussian_blur(image)
            augmentations["blur"].append(blurred_image.unsqueeze(0))
            reduced_brightness = TF.adjust_brightness(image, brightness_factor=0.85)
            augmentations["brightness_reduced"].append(reduced_brightness.unsqueeze(0))
            increased_brightness = TF.adjust_brightness(image, brightness_factor=1.15)
            augmentations["brightness_increased"].append(increased_brightness.unsqueeze(0))

        except Exception as e:
            print(f"Error processing image {image_path}: {e}")

    return augmentations


def calculate_image_cosine_similarity(original_images, augmented_images):
    """
    Calculate cosine similarity directly between original and augmented images.

    Args:
        original_images (list): List of tensors for original images.
        augmented_images (list): List of tensors for augmented images.

    Returns:
        dict: Cosine similarities, differences, and average differences.
    """
    original_images = torch.cat(original_images, dim=0)
    augmented_images = torch.cat(augmented_images, dim=0)

    # Flatten image tensors for cosine similarity computation
    original_images = original_images.view(original_images.size(0), -1)
    augmented_images = augmented_images.view(augmented_images.size(0), -1)

    # Normalize image tensors
    original_images = F.normalize(original_images, p=2, dim=1)
    augmented_images = F.normalize(augmented_images, p=2, dim=1)

    # Compute cosine similarities
    similarities = torch.sum(original_images * augmented_images, dim=1).cpu().tolist()

    # Compute differences and averages
    differences = [1 - sim for sim in similarities]

    return {
        "similarities": similarities,
        "differences": differences,
        "average_difference": sum(differences) / len(differences),
    }


# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_dir = r"C:\\Users\\k54739\\Bibi_new_thesis\\thesis\\crop_simclr\\all"

# Preprocess images with augmentations
augmentations = preprocess_images_from_directory(image_dir)

# Calculate cosine similarities and differences for all augmentations
results = {}
for aug_name, augmented_images in augmentations.items():
    if aug_name != "original":
        results[aug_name] = calculate_image_cosine_similarity(augmentations["original"], augmented_images)

# Print results
for aug_name, result in results.items():
    print(f"{aug_name.capitalize()} Augmentation:")
    print("  Similarities:", result["similarities"])
    print("  Differences (1 - similarity):", result["differences"])
    print("  Average Difference:", result["average_difference"])
    print()

# Task 1: Calculate average of veri and hori augmentations
if "veri" in results and "hori" in results:
    avg_veri_hori = (results["veri"]["average_difference"] + results["hori"]["average_difference"]) / 2
    print(f"Average Difference between Veri and Hori Augmentation: {avg_veri_hori}")

# Task 2: Calculate average of blur and sharpness augmentations
if "blur" in results and "sharpness" in results:
    avg_blur_sharpness = (results["blur"]["average_difference"] + results["sharpness"]["average_difference"]) / 2
    print(f"Average Difference between Blur and Sharpness Augmentation: {avg_blur_sharpness}")

# Task 3: Calculate average of brightness increased and decreased augmentations
if "brightness_increased" in results and "brightness_reduced" in results:
    avg_brightness = (results["brightness_increased"]["average_difference"] + results["brightness_reduced"]["average_difference"]) / 2
    print(f"Average Difference between Brightness Increased and Decreased Augmentation: {avg_brightness}")


Hori Augmentation:
  Similarities: [0.9679598212242126, 0.9883403182029724, 0.9846935868263245, 0.9924450516700745, 0.9836703538894653, 0.9953345060348511, 0.9916730523109436, 0.9890632033348083, 0.9834250807762146, 0.9733330011367798, 0.9857090711593628, 0.9885550737380981, 0.9951069951057434, 0.9826352596282959, 0.9558079242706299, 0.9511807560920715, 0.9491682052612305, 0.927753746509552, 0.9676215648651123, 0.9683417081832886, 0.9607607126235962, 0.9726324677467346, 0.9687578082084656, 0.9744898676872253, 0.9837533235549927, 0.9856339693069458, 0.9587584733963013, 0.9857578277587891, 0.9918637871742249, 0.9785542488098145, 0.9757168889045715, 0.9948300123214722, 0.9743772745132446, 0.9952027797698975, 0.9881975650787354, 0.987197756767273, 0.9955973029136658, 0.9920680522918701, 0.9963234663009644, 0.9954103827476501, 0.9903429746627808, 0.9945558905601501, 0.9848822951316833, 0.9793376922607422, 0.9956852197647095, 0.9848558902740479, 0.9720011949539185, 0.9583483934402466, 0.9902